# P. Дельта-метод

На тех же данных теперь проверим гипотезу равенства среднего чека.

Посчитайте T-статистику, найдите P-value.


Используйте данные из файла [synthetic_gmv_data_1.2.csv](https://github.com/dakhakimova/YSDA_ABweek/blob/476cbc4a49e1f4dfcdb376d69239b6103fbad932/synthetic_gmv_data_1.2.csv)

## Формат вывода


В ответе выведите два числа через пробел.
Целую и дробную часть разделяйте точкой.

Пример ответа: -0.046 0.476

## Примечание

Округлите до 3-го знака после точки.

Используте распределение Стьюдента (вместо нормального), для степеней свободы используйте упрощенную формулу: $n+m−2$ (количество уников теста + количество уников контроля - 2).
Оценивайте дисперсию, используя Дельта-метод.



# Решение

In [3]:
import pandas as pd
import scipy.stats as stats
import numpy as np

In [4]:
url =                "https://github.com/dakhakimova/YSDA_ABweek/blob/476cbc4a49e1f4dfcdb376d69239b6103fbad932/synthetic_gmv_data_1.2.csv"
url = "https://raw.githubusercontent.com/dakhakimova/YSDA_ABweek/" + "476cbc4a49e1f4dfcdb376d69239b6103fbad932/synthetic_gmv_data_1.2.csv"
df = pd.read_csv(url)

In [5]:
df.head()

,user_id,gmv,group_name
0,myo4ixol31,1428,test
1,myo4ixol31,1428,test
2,myo4ixol31,1071,test
3,myo4ixol31,1071,test
4,pkzf2889ww,351,test


In [6]:
df.size

2397183

In [7]:
df['group_name'].unique()

array(['test', 'control'], dtype=object)

In [8]:
test = df[df['group_name'] == 'test']
control = df[df['group_name'] == 'control']

In [9]:
X_test = test.groupby('user_id')['gmv'].sum()
X_control = control.groupby('user_id')['gmv'].sum()

In [10]:
Y_test = test.value_counts('user_id')
Y_control = control.value_counts('user_id')

In [11]:
Y_test.head()

user_id
jfe78xzcq2    15
xqoheyv3i6    14
uhvljh9dfu    14
cvhmnm1kv1    14
lv6kok1ro4    14
Name: count, dtype: int64

In [12]:
value, pvalue = stats.ttest_ind(X_test/Y_test, X_control/Y_control, equal_var=False)
value, pvalue

(2.971077027899215, 0.002968417069164117)

In [13]:
round(value,3), round(pvalue,3)

(2.971, 0.003)

### Дельта-метод

In [14]:
def delta_var(numerator: pd.Series, denominator: pd.Series):
    """
    Функция для расчета дисперсии дельта-методом, numerator - вектор числитель, denominator - вектор знаменатель
    """
    x = numerator
    y = denominator
    n = len(x)
    mu_x = x.mean()
    mu_y = y.mean()
    var_x = x.var(ddof=1)
    var_y = y.var(ddof=1)
    cov_xy = x.cov(y, ddof=1)
    delta_var = ((var_x/mu_y**2)  - 2*cov_xy*(mu_x/mu_y**3) + var_y*(mu_x**2/mu_y**4)) / n
    return delta_var


In [15]:
test_var = delta_var(X_test, Y_test)
control_var = delta_var(X_control, Y_control)
test_var, control_var

(2.1727173707917653, 0.714227879688477)

In [16]:
mean_test = X_test.mean()
mean_control = X_control.mean()
mean_test, mean_control

(2870.650366598778, 2847.2169990019756)

In [17]:
n_test = len(X_test)
n_control = len(X_control)
n_test, n_control

(49100, 147291)

In [18]:
T = (mean_test - mean_control) / np.sqrt(test_var/n_test + control_var/n_control)
T

3344.2146910236224

In [73]:
pow = n_test + n_control - 2
p_value = 2 * stats.t.sf(abs(T), pow)
p_value

2.998669648217774e-43